<a href="https://colab.research.google.com/github/TanzeelAbbas/Assignment-Repo/blob/main/Bert_Questions_Answers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Simple Transformers**
This library is based on the Transformers library by HuggingFace.

In [23]:
import pandas as pd

df = pd.read_json("/content/train-v2.0.json")

print(df.head())

  version                                               data
0    v2.0  {'title': 'Beyoncé', 'paragraphs': [{'qas': [{...
1    v2.0  {'title': 'Frédéric_Chopin', 'paragraphs': [{'...
2    v2.0  {'title': 'Sino-Tibetan_relations_during_the_M...
3    v2.0  {'title': 'IPod', 'paragraphs': [{'qas': [{'qu...
4    v2.0  {'title': 'The_Legend_of_Zelda:_Twilight_Princ...


In [26]:
df[:2]['data']

0    {'title': 'Beyoncé', 'paragraphs': [{'qas': [{...
1    {'title': 'Frédéric_Chopin', 'paragraphs': [{'...
Name: data, dtype: object

In [1]:
# !pip install datasets

from datasets import load_dataset

dataset = load_dataset("squad_v2")

In [2]:
train_dataset = dataset['train']
validation_dataset = dataset['validation']

print(type(train_dataset))
print(type(validation_dataset))

<class 'datasets.arrow_dataset.Dataset'>
<class 'datasets.arrow_dataset.Dataset'>


## **Access the first data point in the training set**

In [3]:
sample = train_dataset[0]
question = sample['question']
context = sample['context']
answers = sample['answers']

print('First Question: ',question)
print("Context: ", context)
print("Answers: ", answers)

First Question:  When did Beyonce start becoming popular?
Context:  Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, the group became one of the world's best-selling girl groups of all time. Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".
Answers:  {'text': ['in the late 1990s'], 'answer_start': [269]}


In [4]:
sample

{'id': '56be85543aeaaa14008c9063',
 'title': 'Beyoncé',
 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'question': 'When did Beyonce start becoming popular?',
 'answers': {'text': ['in the late 1990s'], 'answer_start': [269]}}

In [5]:
print("Train_dataset Length: ", len(train_dataset))
print("Validation_dataset Length: ", len(validation_dataset))

Train_dataset Length:  130319
Validation_dataset Length:  11873


# **Convert the datasets to lists**

In [6]:

trainDataset = []
for example in train_dataset:
    question = example['question']
    context = example['context']
    answers = example.get('answers', [])  # Use .get() to handle missing answers

    qas = []  # Initialize an empty list for answers

    # Check if there are answers
    if answers:
        for i, answer_text in enumerate(answers['text']):
            answer_start = answers['answer_start'][i]
            qas.append({'question': question, 'id': example['id'], 'is_impossible': False, 'answers': [{'text': answer_text, 'answer_start': answer_start}]})
    else:
        # Handle unanswerable questions
        qas = [{'question': question, 'id': example['id'], 'is_impossible': True, 'answers': []}]

    trainDataset.append({'context': context, 'qas': qas})


In [7]:

valDataset = []

for example in validation_dataset:
    question = example['question']
    context = example['context']
    answers = example.get('answers', [])  # Use .get() to handle missing answers

    # Check if there are answers
    if answers:
        qas = [{'question': question, 'id': example['id'], 'is_impossible': False, 'answers': answers}]
    else:
        # Handle unanswerable questions
        qas = [{'question': question, 'id': example['id'], 'is_impossible': True, 'answers': []}]

    valDataset.append({'context': context, 'qas': qas})


In [8]:
print("Train_dataset Length: ", len(trainDataset))
print("Validation_dataset Length: ", len(valDataset))
print("----------------------------------------------------")
valDataset[0]

Train_dataset Length:  130319
Validation_dataset Length:  11873
----------------------------------------------------


{'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.',
 'qas': [{'question': 'In what country is Normandy located?',
   'id': '56ddde6b9a695914005b9628',
   'is_impossible': False,
   'answers': {'text': ['France', 'France', 'France', 'France'],
    'answer_start': [159, 159, 159, 159]}}]}

In [9]:
import random

random.seed(5)

# Select a random subset of 1000 training samples
train_subset = random.sample(trainDataset, 1000)

# Select a random subset of 300 validation samples
val_subset = random.sample(valDataset, 300)

In [10]:
random.seed(23)
test_subset = random.sample(valDataset, 200)

In [26]:
import json
# Save the training subset to a JSON file
with open('train_subset.json', 'w') as train_file:
    json.dump(train_subset, train_file)

# Save the validation subset to a JSON file
with open('val_subset.json', 'w') as val_file:
    json.dump(val_subset, val_file)

# Save the test subset to a JSON file
with open('test_subset.json', 'w') as test_file:
    json.dump(test_subset, test_file)

In [11]:
print("Train_dataset Length: ", len(train_subset))
print("Validation_dataset Length: ", len(val_subset))
print("test_dataset Length: ", len(test_subset))

Train_dataset Length:  1000
Validation_dataset Length:  300
test_dataset Length:  200


# **Import and Set perameters**

In [12]:
# !pip install simpletransformers

import logging
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

model_type= 'bert'
model_name= 'bert-base-uncased'

# **Configure the model**

In [84]:
model_args = QuestionAnsweringArgs()

model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.n_best_size= 3
model_args.num_train_epochs=10

In [15]:
### Advanced Methodology
# ! pip install wandb

train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"/content/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 2,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":2,
    # "use_early_stopping": True,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # }
}

# **Initialize Model**

In [14]:
model = QuestionAnsweringModel(
                  model_type,   # 'bert'
                  model_name,   # 'bert-base-uncased'
                  args= train_args  # model config Arguments
                  )

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.train_model(train_subset, eval_data=val_subset)

add example index and unique id: 100%|██████████| 683/683 [00:00<00:00, 543794.54it/s]


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

In [ ]:
# Define a function to format your dataset
def format_dataset(context, questions, answers):
    dataset = []
    for i, context_text in enumerate(context):
        qa_pairs = []
        for j, question_text in enumerate(questions[i]):
            answer_text = answers[i][j]

            # Create a dictionary for each question-answer pair
            qa_pair = {
                'id': f'{i}_{j}',  # Unique identifier for the pair
                'question': question_text,
                'is_impossible': False if answer_text else True,  # True for unanswerable questions
                'answers': [{'text': answer_text}] if answer_text else [],
            }

            qa_pairs.append(qa_pair)

        # Create a dictionary for the context and its associated question-answer pairs
        context_data = {
            'context': context_text,
            'qas': qa_pairs,
        }

        dataset.append(context_data)

    return dataset

trainDataset=[]

for example in train_dataset:
    questions = example['question']
    context = example['context']
    answers = example['answers']
    trainDataset.append({'question' : question, 'context': context, 'answers':answers})
# Format the dataset
TrainDataset = format_dataset(context, questions, answers)